# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import gamma, digamma, polygamma
import math

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import random

import scipy.optimize
from scipy.stats import ttest_ind
from scipy.optimize import Bounds
from sklearn.preprocessing import StandardScaler

from prettytable import PrettyTable

In [3]:
import multiprocessing

In [4]:
multiprocessing.cpu_count()

8

In [4]:
import dirichlet_regression

In [77]:
# sur mon ordi perso:
scipy.__version__

'1.4.1'

In [5]:
def cos_similarity(x1,x2):
    return(np.mean([np.dot(x1[i],x2[i])/(np.linalg.norm(x1[i])*np.linalg.norm(x2[i])) for i in range(len(x1))]))

In [41]:
from texttable import Texttable
import latextable

# Loading Dataset

In [6]:
arctic = pd.read_csv('Data Dirichlet/ArcticLake.csv')

In [7]:
Y_arctic = arctic.iloc[:,:3]
X_arctic = arctic.iloc[:,3]

In [8]:
Y_arctic = np.array(Y_arctic)

In [9]:
X_arctic = np.array([[j] for j in X_arctic])

In [10]:
Z_arctic = np.ones(len(X_arctic)).reshape((39,1))

In [11]:
n_features = 1
n_classes = 3

In [91]:
X = np.array(X_arctic)
Y = np.array(Y_arctic)

X = StandardScaler().fit(X).transform(X)

In [15]:
Z = Z_arctic
gamma_0 = [0.]

In [16]:
n,K = X.shape
J = Y.shape[1]

# Order 1

In [88]:
distance_matrix = scipy.spatial.distance_matrix(X_arctic,X_arctic)
W_arctic_cont = np.zeros(np.shape(distance_matrix))
W_arctic_cont[distance_matrix < 28] = 1
# replace the 1 on the diagonal by 0
np.fill_diagonal(W_arctic_cont,0)
# scaling the matrix, so that the sum of each row is 1
W_arctic_cont = W_arctic_cont/W_arctic_cont.sum(axis=1)[:,None]

In [89]:
neighbors = NearestNeighbors(n_neighbors=29).fit(X_arctic)
W_arctic_dist = neighbors.kneighbors_graph(X_arctic,mode='distance').toarray()
W_arctic_dist[W_arctic_dist>0] = 1/W_arctic_dist[W_arctic_dist>0]
W_arctic_dist = W_arctic_dist/W_arctic_dist.sum(axis=1)[:,None]

## Dirichlet model

In [92]:
%%time
list_r2_ns_1, list_rmse_ns_1, list_aic_ns_1, list_crossentropy_ns_1, list_similarity_ns_1 = [], [], [], [], []
list_r2_cont_1, list_rmse_cont_1, list_aic_cont_1, list_crossentropy_cont_1, list_similarity_cont_1 = [], [], [], [], []
list_r2_dist_1, list_rmse_dist_1, list_aic_dist_1, list_crossentropy_dist_1, list_similarity_dist_1 = [], [], [], [], []

for i in range(39):  
    X_temp = np.delete(X,i,axis=0)
    Y_temp = np.delete(Y_arctic,i,axis=0)
    Z_temp = np.delete(Z_arctic,i,axis=0)
    
    Wss_cont = np.delete(W_arctic_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_arctic_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp)
    list_r2_ns_1.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_ns_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_ns_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_ns_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*5)
    list_similarity_ns_1.append(cos_similarity(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont)
    list_r2_cont_1.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_cont_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_cont_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_cont_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
    list_similarity_cont_1.append(cos_similarity(Y_temp,dirichRegressor.mu))
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist)
    list_r2_dist_1.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_dist_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_dist_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_dist_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
    list_similarity_dist_1.append(cos_similarity(Y_temp,dirichRegressor.mu))

Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimiza

In [93]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_1),4))+" ("+str(np.round(np.std(list_r2_ns_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1),4))+" ("+str(np.round(np.std(list_rmse_ns_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1),4))+" ("+str(np.round(np.std(list_similarity_ns_1),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_1),4))+" ("+str(np.round(np.std(list_r2_cont_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1),4))+" ("+str(np.round(np.std(list_rmse_cont_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1),4))+" ("+str(np.round(np.std(list_similarity_cont_1),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_1),4))+" ("+str(np.round(np.std(list_r2_dist_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1),4))+" ("+str(np.round(np.std(list_rmse_dist_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1),4))+" ("+str(np.round(np.std(list_similarity_dist_1),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.5887 (0.015) | 0.1015        | 0.9134        | 0.9665        |
|             |                | (0.0018)      | (0.0027)      | (0.0012)      |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.6323         | 0.0951        | 0.9064        | 0.9706        |
|             | (0.0147)       | (0.0017)      | (0.0029)      | (0.0011)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.5893         | 0.1014        | 0.9134        | 0.9665        |
|             | (0.0148)       | (0.0018)      | (0.0027)      | (0.0012)      |
+-------------+-------------

## Cross-entropy

In [40]:
%%time
list_r2_ns_1_ce, list_rmse_ns_1_ce, list_crossentropy_ns_1_ce, list_similarity_ns_1_ce = [], [], [], []
list_r2_cont_1_ce, list_rmse_cont_1_ce, list_crossentropy_cont_1_ce, list_similarity_cont_1_ce = [], [], [], []
list_r2_dist_1_ce, list_rmse_dist_1_ce, list_crossentropy_dist_1_ce, list_similarity_dist_1_ce = [], [], [], []

for i in range(39):  
    X_temp = np.delete(X_arctic,i,axis=0)
    Y_temp = np.delete(Y_arctic,i,axis=0)
    Z_temp = np.delete(Z_arctic,i,axis=0)
    
    Wss_cont = np.delete(W_arctic_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_arctic_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, loss='crossentropy')
    list_r2_ns_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_ns_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_ns_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_ns_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont, loss='crossentropy')
    list_r2_cont_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_cont_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_cont_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_cont_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist, loss='crossentropy')
    list_r2_dist_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_dist_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_dist_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_dist_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))

Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization

In [50]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_1_ce),4))+" ("+str(np.round(np.std(list_r2_ns_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_1_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_1_ce),4))+" ("+str(np.round(np.std(list_r2_cont_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_1_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_1_ce),4))+" ("+str(np.round(np.std(list_r2_dist_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_1_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.6275         | 0.0961        | 0.9106        | 0.97 (0.001)  |
|             | (0.0132)       | (0.0016)      | (0.0027)      |               |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.667 (0.0135) | 0.0899        | 0.9025        | 0.9735        |
|             |                | (0.0018)      | (0.003)       | (0.0011)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.6325         | 0.0946        | 0.9087        | 0.971         |
|             | (0.0132)       | (0.0018)      | (0.0026)      | (0.0012)      |
+-------------+-------------

# Order 2

## Dirichlet model

In [95]:
%%time
list_r2_ns_2, list_rmse_ns_2, list_aic_ns_2, list_crossentropy_ns_2, list_similarity_ns_2 = [], [], [], [], []
list_r2_cont_2, list_rmse_cont_2, list_aic_cont_2, list_crossentropy_cont_2, list_similarity_cont_2 = [], [], [], [], []
list_r2_dist_2, list_rmse_dist_2, list_aic_dist_2, list_crossentropy_dist_2, list_similarity_dist_2 = [], [], [], [], []

for i in range(39):  
    X_temp = np.delete(X_2,i,axis=0)
    Y_temp = np.delete(Y_arctic,i,axis=0)
    Z_temp = np.delete(Z_arctic,i,axis=0)
    
    Wss_cont = np.delete(W_arctic_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_arctic_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp)
    list_r2_ns_2.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_ns_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_ns_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_ns_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*5)
    list_similarity_ns_2.append(cos_similarity(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont)
    list_r2_cont_2.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_cont_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_cont_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_cont_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
    list_similarity_cont_2.append(cos_similarity(Y_temp,dirichRegressor.mu))
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist)
    list_r2_dist_2.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_dist_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_dist_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_aic_dist_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
    list_similarity_dist_2.append(cos_similarity(Y_temp,dirichRegressor.mu))

Optimization terminated successfully.
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization terminated successfully.
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Optimization te

In [96]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_2),4))+" ("+str(np.round(np.std(list_r2_ns_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_2),4))+" ("+str(np.round(np.std(list_rmse_ns_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_2),4))+" ("+str(np.round(np.std(list_crossentropy_ns_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_2),4))+" ("+str(np.round(np.std(list_similarity_ns_2),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_2),4))+" ("+str(np.round(np.std(list_r2_cont_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_2),4))+" ("+str(np.round(np.std(list_rmse_cont_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_2),4))+" ("+str(np.round(np.std(list_crossentropy_cont_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_2),4))+" ("+str(np.round(np.std(list_similarity_cont_2),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_2),4))+" ("+str(np.round(np.std(list_r2_dist_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_2),4))+" ("+str(np.round(np.std(list_rmse_dist_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_2),4))+" ("+str(np.round(np.std(list_crossentropy_dist_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_2),4))+" ("+str(np.round(np.std(list_similarity_dist_2),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.6784         | 0.0881        | 0.8993        | 0.9743        |
|             | (0.0144)       | (0.0019)      | (0.0032)      | (0.0011)      |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.6943         | 0.0858        | 0.8974        | 0.9755        |
|             | (0.0144)       | (0.0019)      | (0.0033)      | (0.0011)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.6863 (0.015) | 0.0871        | 0.8982        | 0.9747        |
|             |                | (0.002)       | (0.0032)      | (0.0012)      |
+-------------+-------------

## Cross-entropy

In [97]:
%%time
list_r2_ns_2_ce, list_rmse_ns_2_ce, list_crossentropy_ns_2_ce, list_similarity_ns_2_ce = [], [], [], []
list_r2_cont_2_ce, list_rmse_cont_2_ce, list_crossentropy_cont_2_ce, list_similarity_cont_2_ce = [], [], [], []
list_r2_dist_2_ce, list_rmse_dist_2_ce, list_crossentropy_dist_2_ce, list_similarity_dist_2_ce = [], [], [], []

for i in range(39):  
    X_temp = np.delete(X_arctic,i,axis=0)
    Y_temp = np.delete(Y_arctic,i,axis=0)
    Z_temp = np.delete(Z_arctic,i,axis=0)
    
    Wss_cont = np.delete(W_arctic_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_arctic_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, loss='crossentropy')
    list_r2_ns_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_ns_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_ns_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_ns_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont, loss='crossentropy')
    list_r2_cont_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_cont_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_cont_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_cont_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist, loss='crossentropy')
    list_r2_dist_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
    list_rmse_dist_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
    list_crossentropy_dist_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
    list_similarity_dist_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))

Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization

In [98]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_1_ce),4))+" ("+str(np.round(np.std(list_r2_ns_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_1_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_1_ce),4))+" ("+str(np.round(np.std(list_r2_cont_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_1_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_1_ce),4))+" ("+str(np.round(np.std(list_r2_dist_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_1_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.6275         | 0.0961        | 0.9106        | 0.97 (0.001)  |
|             | (0.0132)       | (0.0016)      | (0.0027)      |               |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.667 (0.0135) | 0.0899        | 0.9025        | 0.9735        |
|             |                | (0.0018)      | (0.003)       | (0.0011)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.6325         | 0.0946        | 0.9087        | 0.971         |
|             | (0.0132)       | (0.0018)      | (0.0026)      | (0.0012)      |
+-------------+-------------